In [40]:
player_cols_to_agg = [
    'AFLfantasy',
    'Supercoach',
    'behinds',
    'bounces',
    'brownlow',
    'clangers',
    'clearances',
    'contested_marks',
    'contested_poss',
    'disposals',
    'frees_against',
    'frees_for',
    'goal_assists',
    'goals',
    'handballs',
    'hitouts',
    'inside50',
    'kicks',
    'marks',
    'marks_in_50',
    'one_percenters',
    'rebound50',
    'tackles',
    'tog',
    'uncontested_poss',
    'centre_clearances',
    'disposal_efficiency',
    'effective_disposals',
    'intercepts',
    'metres_gained',
    'stoppage_clearances',
    'score_involvements',
    'tackles_in_50',
    'turnovers'
]

match_cols = [
    'odds',
    'line'
]

ladder_columns = [
    'wins',
    'losses',
    'draws',
    'prem_points',
    'played',
    'points_for',
    'points_against',
    'percentage',
    'position'
]

misc_columns = [
    'games_played'
]

In [41]:
# Check team results
import pandas as pd
team_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/matches_with_player_agg.csv'
team_df = pd.read_csv(team_path)

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (19) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [42]:
team_df

,Unnamed: 0,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,...,a_season_disposal_efficiency,a_season_effective_disposals,a_season_intercepts,a_season_metres_gained,a_season_stoppage_clearances,a_season_score_involvements,a_season_tackles_in_50,a_season_turnovers,a_career_games_played,a_season_games_played
0,0,1,Victoria Park,1897-05-08,Sat,15:00:00,NaN,Collingwood,1.1.7,4.3.27,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,20
1,1,1,Brunswick St,1897-05-08,Sat,15:00:00,3000.0,Fitzroy,0.5.5,0.11.11,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,20
2,2,1,Lake Oval,1897-05-08,Sat,15:00:00,NaN,South Melbourne,0.2.2,1.4.10,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,20
3,3,1,Corio Oval,1897-05-08,Sat,15:00:00,NaN,Geelong,0.0.0,0.4.4,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,20,20
4,4,2,East Melbourne,1897-05-15,Sat,15:00:00,NaN,Essendon,1.3.9,1.5.11,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,39,39
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14927,14927,23,Bellerive Oval,2019-08-24,Sat,14:10:00,8202.0,North Melbourne,3.4.22,5.4.34,...,1516.885457,4519.0,1245.0,100184.0,527.0,1463.0,189.0,1321.0,2021,356
14928,14928,23,S.C.G.,2019-08-24,Sat,13:45:00,33722.0,Sydney,6.2.38,6.3.39,...,1605.500164,4548.0,1100.0,93830.0,512.0,1518.0,175.0,1116.0,2095,363
14929,14929,23,Eureka Stadium,2019-08-25,Sun,13:10:00,9560.0,Footscray,6.5.41,9.8.62,...,1514.474015,4607.0,1176.0,88356.0,422.0,1464.0,141.0,1133.0,2809,336
14930,14930,23,Adelaide Oval,2019-08-25,Sun,16:10:00,27504.0,Port Adelaide,3.4.22,7.9.51,...,1535.230924,4358.0,1155.0,91921.0,474.0,1424.0,151.0,1203.0,1643,344


In [43]:
available_date = {}
for col in player_cols_to_agg:
    h_col = 'h_' + col
    first = team_df[team_df[h_col] != 0]['date'].iloc[0]
    first_season = int(first[0:4])
    available_date[col] = first_season

In [44]:
# Need a function to tell us what the biggest predictors of a victory are. Essentially the test will be to see
# how often the team that wins each category turns out to be the winner
def win_corr(cols):
    
    corr = []
    
    for col in cols:
        if col in ladder_columns: # Ladder column conditions
            h_col = 'h_' + col
            a_col = 'a_' + col
            h_col_form = 'h_' + col + '_form'
            a_col_form = 'a_' + col + '_form'
            #first_season = available_date[col]
            first_season = 1897

            relevant_games = team_df[(team_df['season'] >= first_season) &
                                    (team_df[a_col] != team_df[h_col])] # Added this to remove games where they are same
            relevant_games_form = team_df[(team_df['season'] >= first_season) &
                                    (team_df[a_col_form] != team_df[h_col_form])]
            
            total_games = len(relevant_games)
            total_games_form = len(relevant_games_form)
            col_corr = (sum((team_df[h_col] > team_df[a_col]) & (team_df['winner'] == 'home')) 
                          + sum((team_df[h_col] < team_df[a_col]) & (team_df['winner'] == 'away'))) \
                          / total_games

            col_corr_form = (sum((team_df[h_col_form] > team_df[a_col_form]) & (team_df['winner'] == 'home'))
                          + sum((team_df[h_col_form] < team_df[a_col_form]) & (team_df['winner'] == 'away'))) \
                          / total_games_form

            corr.append((col, col_corr))
            corr.append((col + '_form', col_corr_form))
            
        if col in player_cols_to_agg: # Aggregated player stats conditions
            h_col_career = 'h_career_' + col
            a_col_career = 'a_career_' + col
            h_col_season = 'h_season_' + col
            a_col_season = 'a_season_' + col
            first_season = available_date[col]

            relevant_games = team_df[team_df['season'] >= first_season]
            total_games = len(relevant_games)
            col_corr_career = (sum((team_df[h_col_career] > team_df[a_col_career]) & (team_df['winner'] == 'home')) 
                          + sum((team_df[h_col_career] < team_df[a_col_career]) & (team_df['winner'] == 'away'))) \
                          / total_games

            col_corr_season = (sum((team_df[h_col_season] > team_df[a_col_season]) & (team_df['winner'] == 'home'))
                          + sum((team_df[h_col_season] < team_df[a_col_season]) & (team_df['winner'] == 'away'))) \
                          / total_games

            corr.append(('career_' + col, col_corr_career))
            corr.append(('season_' + col, col_corr_season))
            
        if col in misc_columns: # For games played
            h_col_career = 'h_career_' + col
            a_col_career = 'a_career_' + col
            h_col_season = 'h_season_' + col
            a_col_season = 'a_season_' + col
            first_season = 1897

            relevant_games = team_df[team_df['season'] >= first_season]
            total_games = len(relevant_games)
            col_corr_career = (sum((team_df[h_col_career] > team_df[a_col_career]) & (team_df['winner'] == 'home')) 
                          + sum((team_df[h_col_career] < team_df[a_col_career]) & (team_df['winner'] == 'away'))) \
                          / total_games

            col_corr_season = (sum((team_df[h_col_season] > team_df[a_col_season]) & (team_df['winner'] == 'home'))
                          + sum((team_df[h_col_season] < team_df[a_col_season]) & (team_df['winner'] == 'away'))) \
                          / total_games

            corr.append(('career_' + col, col_corr_career))
            corr.append(('season_' + col, col_corr_season))
    
    return corr

In [45]:
corr = win_corr(ladder_columns + player_cols_to_agg + misc_columns)

In [46]:
corr.sort(key = lambda x: x[1])
corr

[('losses', 0.31657528469000423),
 ('position', 0.3312658584719481),
 ('points_against', 0.35268330622923),
 ('played_form', 0.4027777777777778),
 ('career_disposal_efficiency', 0.41234439834024894),
 ('losses_form', 0.41586058038212487),
 ('points_against_form', 0.42893639908565284),
 ('position_form', 0.47333592837680033),
 ('draws_form', 0.4853801169590643),
 ('season_rebound50', 0.49554455445544554),
 ('draws', 0.5094643576318969),
 ('percentage_form', 0.5101165751038457),
 ('season_clangers', 0.5193069306930693),
 ('season_frees_for', 0.5224036123654047),
 ('season_turnovers', 0.5272914521112255),
 ('career_hitouts', 0.5331222886622112),
 ('season_hitouts', 0.5371086880056278),
 ('season_bounces', 0.5403726708074534),
 ('points_for_form', 0.5520854321982672),
 ('season_games_played', 0.5577953388695419),
 ('career_bounces', 0.5610766045548654),
 ('season_centre_clearances', 0.568486096807415),
 ('played', 0.5706106870229007),
 ('season_frees_against', 0.5709158272548338),
 ('prem_

In [47]:
corr.sort(key = lambda x: x[1])
corr

[('losses', 0.31657528469000423),
 ('position', 0.3312658584719481),
 ('points_against', 0.35268330622923),
 ('played_form', 0.4027777777777778),
 ('career_disposal_efficiency', 0.41234439834024894),
 ('losses_form', 0.41586058038212487),
 ('points_against_form', 0.42893639908565284),
 ('position_form', 0.47333592837680033),
 ('draws_form', 0.4853801169590643),
 ('season_rebound50', 0.49554455445544554),
 ('draws', 0.5094643576318969),
 ('percentage_form', 0.5101165751038457),
 ('season_clangers', 0.5193069306930693),
 ('season_frees_for', 0.5224036123654047),
 ('season_turnovers', 0.5272914521112255),
 ('career_hitouts', 0.5331222886622112),
 ('season_hitouts', 0.5371086880056278),
 ('season_bounces', 0.5403726708074534),
 ('points_for_form', 0.5520854321982672),
 ('season_games_played', 0.5577953388695419),
 ('career_bounces', 0.5610766045548654),
 ('season_centre_clearances', 0.568486096807415),
 ('played', 0.5706106870229007),
 ('season_frees_against', 0.5709158272548338),
 ('prem_

Note that a lot of these come from the players, and therefore might need some tweaking. Rather than season and career, perhaps season per game and career per game would be better.

Supercoach is insanely predictive!

In [48]:
# Need to fix this up to not divide by games where the metric is the same

from collections import defaultdict

metric_dict = defaultdict(lambda: 0)

for metric in [tup[0] for tup in corr]:
    metric_dict[metric] = defaultdict(lambda: 0)

for idx, row in team_df.iterrows():
    print(idx/len(team_df))
    
    for metric in metric_dict:
        
        try:
            if ((row['h_' + metric] > row['a_' + metric]) & (row['winner'] == 'home')) | \
                ((row['h_' + metric] < row['a_' + metric]) & (row['winner'] == 'away')):
                metric_dict[metric][row['season']] += 1
        except KeyError:
            pass

for metric in metric_dict:
    for season, value in metric_dict[metric].items():
        try:
            metric_dict[metric][season] = value/sum((team_df['season']==season) &
                                                    (team_df['h_' + metric] != team_df['a_' + metric]))
        except KeyError:
            pass

0.0
6.69702652022502e-05
0.0001339405304045004
0.0002009107956067506
0.0002678810608090008
0.000334851326011251
0.0004018215912135012
0.0004687918564157514
0.0005357621216180017
0.0006027323868202519
0.000669702652022502
0.0007366729172247522
0.0008036431824270024
0.0008706134476292526
0.0009375837128315028
0.001004553978033753
0.0010715242432360033
0.0011384945084382534
0.0012054647736405037
0.0012724350388427538
0.001339405304045004
0.0014063755692472542
0.0014733458344495043
0.0015403160996517546
0.0016072863648540047
0.001674256630056255
0.0017412268952585051
0.0018081971604607555
0.0018751674256630056
0.0019421376908652559
0.002009107956067506
0.0020760782212697563
0.0021430484864720066
0.0022100187516742565
0.002276989016876507
0.002343959282078757
0.0024109295472810074
0.0024778998124832573
0.0025448700776855076
0.002611840342887758
0.002678810608090008
0.002745780873292258
0.0028127511384945084
0.0028797214036967588
0.0029466916688990086
0.003013661934101259
0.00308063219930350

0.028931154567372087
0.028998124832574336
0.029065095097776588
0.029132065362978837
0.02919903562818109
0.029266005893383337
0.02933297615858559
0.029399946423787838
0.02946691668899009
0.02953388695419234
0.029600857219394587
0.02966782748459684
0.029734797749799088
0.02980176801500134
0.02986873828020359
0.02993570854540584
0.03000267881060809
0.03006964907581034
0.03013661934101259
0.03020358960621484
0.03027055987141709
0.030337530136619342
0.03040450040182159
0.030471470667023843
0.03053844093222609
0.030605411197428344
0.030672381462630592
0.03073935172783284
0.030806321993035093
0.03087329225823734
0.030940262523439593
0.031007232788641842
0.031074203053844094
0.031141173319046343
0.031208143584248595
0.03127511384945084
0.03134208411465309
0.03140905437985535
0.031476024645057596
0.031542994910259844
0.03160996517546209
0.03167693544066434
0.0317439057058666
0.031810875971068846
0.031877846236271094
0.03194481650147334
0.0320117867666756
0.03207875703187785
0.032145727297080096

0.05712563621751942
0.05719260648272167
0.057259576747923924
0.05732654701312617
0.05739351727832842
0.05746048754353067
0.057527457808732925
0.057594428073935174
0.05766139833913742
0.05772836860433967
0.057795338869541926
0.057862309134744175
0.05792927939994642
0.05799624966514867
0.05806321993035093
0.058130190195553176
0.058197160460755425
0.05826413072595767
0.05833110099115992
0.05839807125636218
0.058465041521564426
0.058532011786766674
0.05859898205196892
0.05866595231717118
0.05873292258237343
0.058799892847575676
0.058866863112777924
0.05893383337798018
0.05900080364318243
0.05906777390838468
0.059134744173586926
0.059201714438789174
0.05926868470399143
0.05933565496919368
0.05940262523439593
0.059469595499598175
0.05953656576480043
0.05960353603000268
0.05967050629520493
0.05973747656040718
0.05980444682560943
0.05987141709081168
0.05993838735601393
0.06000535762121618
0.06007232788641843
0.06013929815162068
0.06020626841682293
0.06027323868202518
0.06034020894722743
0.0604

0.08598982051968926
0.08605679078489151
0.08612376105009376
0.086190731315296
0.08625770158049825
0.08632467184570052
0.08639164211090276
0.08645861237610501
0.08652558264130726
0.08659255290650951
0.08665952317171176
0.086726493436914
0.08679346370211626
0.0868604339673185
0.08692740423252077
0.08699437449772301
0.08706134476292526
0.08712831502812751
0.08719528529332976
0.08726225555853201
0.08732922582373426
0.0873961960889365
0.08746316635413877
0.08753013661934102
0.08759710688454327
0.08766407714974551
0.08773104741494776
0.08779801768015001
0.08786498794535226
0.08793195821055451
0.08799892847575676
0.08806589874095902
0.08813286900616127
0.08819983927136352
0.08826680953656577
0.08833377980176801
0.08840075006697026
0.08846772033217251
0.08853469059737476
0.08860166086257702
0.08866863112777927
0.08873560139298152
0.08880257165818377
0.08886954192338602
0.08893651218858827
0.08900348245379051
0.08907045271899276
0.08913742298419501
0.08920439324939727
0.08927136351459952
0.0893

0.11605946959549959
0.11612643986070185
0.1161934101259041
0.11626038039110635
0.1163273506563086
0.11639432092151085
0.1164612911867131
0.11652826145191535
0.1165952317171176
0.11666220198231984
0.1167291722475221
0.11679614251272435
0.1168631127779266
0.11693008304312885
0.1169970533083311
0.11706402357353335
0.1171309938387356
0.11719796410393785
0.11726493436914011
0.11733190463434236
0.1173988748995446
0.11746584516474685
0.1175328154299491
0.11759978569515135
0.1176667559603536
0.11773372622555585
0.1178006964907581
0.11786766675596036
0.11793463702116261
0.11800160728636486
0.1180685775515671
0.11813554781676935
0.1182025180819716
0.11826948834717385
0.1183364586123761
0.11840342887757835
0.11847039914278061
0.11853736940798286
0.11860433967318511
0.11867130993838736
0.1187382802035896
0.11880525046879185
0.1188722207339941
0.11893919099919635
0.11900616126439861
0.11907313152960086
0.11914010179480311
0.11920707206000536
0.11927404232520761
0.11934101259040986
0.119407982855612

...

0.8625770158049826
0.8626439860701849
0.8627109563353871
0.8627779266005894
0.8628448968657916
0.8629118671309939
0.8629788373961961
0.8630458076613984
0.8631127779266006
0.8631797481918029
0.8632467184570051
0.8633136887222074
0.8633806589874096
0.8634476292526119
0.8635145995178141
0.8635815697830164
0.8636485400482186
0.8637155103134209
0.8637824805786231
0.8638494508438254
0.8639164211090276
0.8639833913742299
0.864050361639432
0.8641173319046344
0.8641843021698365
0.8642512724350389
0.864318242700241
0.8643852129654434
0.8644521832306455
0.8645191534958478
0.86458612376105
0.8646530940262523
0.8647200642914546
0.8647870345566568
0.8648540048218591
0.8649209750870613
0.8649879453522636
0.8650549156174658
0.8651218858826681
0.8651888561478703
0.8652558264130726
0.8653227966782748
0.8653897669434771
0.8654567372086793
0.8655237074738816
0.8655906777390838
0.8656576480042861
0.8657246182694883
0.8657915885346906
0.8658585587998928
0.8659255290650951
0.8659924993302973
0.86605946959549

0.8925127243503884
0.8925796946155907
0.8926466648807929
0.8927136351459952
0.8927806054111974
0.8928475756763997
0.8929145459416019
0.8929815162068042
0.8930484864720064
0.8931154567372087
0.8931824270024109
0.8932493972676132
0.8933163675328154
0.8933833377980177
0.8934503080632199
0.8935172783284222
0.8935842485936244
0.8936512188588267
0.8937181891240289
0.8937851593892312
0.8938521296544334
0.8939190999196357
0.8939860701848379
0.8940530404500402
0.8941200107152424
0.8941869809804447
0.8942539512456469
0.8943209215108492
0.8943878917760514
0.8944548620412537
0.894521832306456
0.8945888025716582
0.8946557728368605
0.8947227431020627
0.894789713367265
0.8948566836324672
0.8949236538976695
0.8949906241628717
0.895057594428074
0.8951245646932762
0.8951915349584785
0.8952585052236807
0.895325475488883
0.8953924457540852
0.8954594160192875
0.8955263862844897
0.895593356549692
0.8956603268148942
0.8957272970800965
0.8957942673452987
0.895861237610501
0.8959282078757032
0.8959951781409055

0.9217117599785695
0.9217787302437718
0.921845700508974
0.9219126707741763
0.9219796410393785
0.9220466113045808
0.9221135815697831
0.9221805518349853
0.9222475221001876
0.9223144923653898
0.9223814626305921
0.9224484328957943
0.9225154031609966
0.9225823734261988
0.9226493436914011
0.9227163139566033
0.9227832842218056
0.9228502544870077
0.92291722475221
0.9229841950174122
0.9230511652826145
0.9231181355478167
0.923185105813019
0.9232520760782212
0.9233190463434235
0.9233860166086257
0.923452986873828
0.9235199571390302
0.9235869274042325
0.9236538976694347
0.923720867934637
0.9237878381998392
0.9238548084650415
0.9239217787302437
0.923988748995446
0.9240557192606482
0.9241226895258505
0.9241896597910527
0.924256630056255
0.9243236003214573
0.9243905705866595
0.9244575408518618
0.924524511117064
0.9245914813822663
0.9246584516474685
0.9247254219126708
0.924792392177873
0.9248593624430753
0.9249263327082775
0.9249933029734798
0.925060273238682
0.9251272435038843
0.9251942137690865
0.92

0.9529199035628181
0.9529868738280204
0.9530538440932226
0.9531208143584249
0.9531877846236271
0.9532547548888294
0.9533217251540316
0.9533886954192339
0.9534556656844361
0.9535226359496384
0.9535896062148406
0.9536565764800429
0.9537235467452451
0.9537905170104474
0.9538574872756496
0.9539244575408519
0.9539914278060541
0.9540583980712564
0.9541253683364586
0.9541923386016609
0.9542593088668632
0.9543262791320654
0.9543932493972677
0.9544602196624699
0.9545271899276722
0.9545941601928744
0.9546611304580767
0.9547281007232788
0.9547950709884812
0.9548620412536833
0.9549290115188857
0.9549959817840878
0.9550629520492901
0.9551299223144923
0.9551968925796946
0.9552638628448968
0.9553308331100991
0.9553978033753013
0.9554647736405036
0.9555317439057058
0.9555987141709081
0.9556656844361103
0.9557326547013126
0.9557996249665148
0.9558665952317171
0.9559335654969193
0.9560005357621216
0.9560675060273238
0.9561344762925261
0.9562014465577283
0.9562684168229306
0.9563353870881328
0.9564023573

0.9839271363514599
0.9839941066166622
0.9840610768818645
0.9841280471470667
0.984195017412269
0.9842619876774712
0.9843289579426735
0.9843959282078757
0.984462898473078
0.9845298687382802
0.9845968390034825
0.9846638092686847
0.984730779533887
0.9847977497990892
0.9848647200642915
0.9849316903294937
0.984998660594696
0.9850656308598982
0.9851326011251005
0.9851995713903027
0.985266541655505
0.9853335119207072
0.9854004821859095
0.9854674524511117
0.985534422716314
0.9856013929815162
0.9856683632467185
0.9857353335119207
0.985802303777123
0.9858692740423252
0.9859362443075275
0.9860032145727297
0.986070184837932
0.9861371551031342
0.9862041253683365
0.9862710956335388
0.986338065898741
0.9864050361639433
0.9864720064291455
0.9865389766943478
0.98660594695955
0.9866729172247523
0.9867398874899544
0.9868068577551568
0.986873828020359
0.9869407982855612
0.9870077685507634
0.9870747388159657
0.9871417090811679
0.9872086793463702
0.9872756496115724
0.9873426198767747
0.9874095901419769
0.987

In [15]:
metric_dict

defaultdict(<function __main__.<lambda>()>,
            {'season_centre_clearances': defaultdict(<function __main__.<lambda>()>,
                         {2015: 0.6162162162162163,
                          2016: 0.6224489795918368,
                          2017: 0.5416666666666666,
                          2018: 0.5714285714285714,
                          2019: 0.4461538461538462}),
             'season_stoppage_clearances': defaultdict(<function __main__.<lambda>()>,
                         {2015: 0.6162162162162163,
                          2016: 0.6153846153846154,
                          2017: 0.5794871794871795,
                          2018: 0.5977653631284916,
                          2019: 0.47715736040609136}),
             'season_turnovers': defaultdict(<function __main__.<lambda>()>,
                         {2015: 0.601063829787234,
                          2016: 0.6615384615384615,
                          2017: 0.5846153846153846,
                          2

In [277]:
metric_dict['wins_form']

defaultdict(<function __main__.<lambda>()>,
            {1897: 0.7804878048780488,
             1898: 0.603448275862069,
             1899: 0.5967741935483871,
             1900: 0.55,
             1901: 0.5409836065573771,
             1902: 0.5245901639344263,
             1903: 0.6440677966101694,
             1904: 0.3888888888888889,
             1905: 0.5087719298245614,
             1906: 0.6,
             1907: 0.5483870967741935,
             1908: 0.5857142857142857,
             1909: 0.6623376623376623,
             1910: 0.6282051282051282,
             1911: 0.6164383561643836,
             1912: 0.5789473684210527,
             1913: 0.6578947368421053,
             1914: 0.5375,
             1915: 0.7368421052631579,
             1916: 0.5652173913043478,
             1917: 0.4146341463414634,
             1918: 0.574468085106383,
             1919: 0.4838709677419355,
             1920: 0.6875,
             1921: 0.53125,
             1922: 0.6,
             1923: 0.5,

In [264]:
metric_dict['season_Supercoach']

defaultdict(<function __main__.<lambda>()>,
            {2010: 0.6988636363636364,
             2011: 0.7433155080213903,
             2012: 0.7626262626262627,
             2013: 0.7323232323232324,
             2014: 0.7373737373737373,
             2015: 0.7461928934010152,
             2016: 0.7878787878787878,
             2017: 0.7272727272727273,
             2018: 0.7277777777777777,
             2019: 0.6515151515151515})

In [265]:
metric_dict['season_goal_assists']

defaultdict(<function __main__.<lambda>()>,
            {2003: 0.6363636363636364,
             2004: 0.6420454545454546,
             2005: 0.6647727272727273,
             2006: 0.6647727272727273,
             2007: 0.6647727272727273,
             2008: 0.6875,
             2009: 0.7045454545454546,
             2010: 0.6534090909090909,
             2011: 0.7058823529411765,
             2012: 0.7323232323232324,
             2013: 0.7373737373737373,
             2014: 0.7424242424242424,
             2015: 0.7309644670050761,
             2016: 0.7373737373737373,
             2017: 0.6919191919191919,
             2018: 0.6666666666666666,
             2019: 0.696969696969697})

In [17]:
# Get player data for lookup
player_path = '/Users/t_raver9/Desktop/projects/aflengine/analysis/machine_learning/src/player_data/data/players_with_player_stat_totals.csv'
player_df = pd.read_csv(player_path)

/Applications/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3058: DtypeWarning: Columns (15,20,27) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


Strange things to investigate:
    
    - wins form is harmful for predicitons. Probably something wrong in the calc.
    - career disposal efficiency is harmful for predictions

In [18]:
team_df[['h_wins_form','a_wins_form','hteam','ateam','season','round']].iloc[-100:]

,h_wins_form,a_wins_form,hteam,ateam,season,round
14832,2,4,North Melbourne,Richmond,2019,11
14833,4,2,Collingwood,Melbourne,2019,12
14834,3,5,Richmond,Geelong,2019,12
14835,0,3,Carlton,Brisbane Lions,2019,12
14836,3,4,Adelaide,Greater Western Sydney,2019,12
...,...,...,...,...,...,...
14927,2,0,North Melbourne,Melbourne,2019,23
14928,1,3,Sydney,St Kilda,2019,23
14929,3,1,Footscray,Adelaide,2019,23
14930,2,2,Port Adelaide,Fremantle,2019,23


In [19]:
# What about how often the favourite wins?

relevant_games = team_df[(team_df['season'] >= 2010) & (team_df['homeodds'] != team_df['awayodds'])]
total_games = len(relevant_games)
odds_corr = (sum((team_df['homeodds'] > team_df['awayodds']) & (team_df['winner'] == 'home')) 
              + sum((team_df['homeodds'] < team_df['awayodds']) & (team_df['winner'] == 'away'))) \
              / total_games

In [20]:
1 - odds_corr

0.7180823345492444

In [21]:
# Does this beat supercoach?
from collections import defaultdict

odds_dict = defaultdict(lambda: 0)

for idx, row in team_df.iterrows():
    print(idx/len(team_df))

    if ((row['homeodds'] > row['awayodds']) & (row['winner'] == 'home')) | \
        ((row['homeodds'] < row['awayodds']) & (row['winner'] == 'away')):
        odds_dict[row['season']] += 1

for season in odds_dict:
    odds_dict[season] = value/sum((team_df['season']==season) & (team_df['homeodds'] != team_df['awayodds']))

0.0
6.69702652022502e-05
0.0001339405304045004
0.0002009107956067506
0.0002678810608090008
0.000334851326011251
0.0004018215912135012
0.0004687918564157514
0.0005357621216180017
0.0006027323868202519
0.000669702652022502
0.0007366729172247522
0.0008036431824270024
0.0008706134476292526
0.0009375837128315028
0.001004553978033753
0.0010715242432360033
0.0011384945084382534
0.0012054647736405037
0.0012724350388427538
0.001339405304045004
0.0014063755692472542
0.0014733458344495043
0.0015403160996517546
0.0016072863648540047
0.001674256630056255
0.0017412268952585051
0.0018081971604607555
0.0018751674256630056
0.0019421376908652559
0.002009107956067506
0.0020760782212697563
0.0021430484864720066
0.0022100187516742565
0.002276989016876507
0.002343959282078757
0.0024109295472810074
0.0024778998124832573
0.0025448700776855076
0.002611840342887758
0.002678810608090008
0.002745780873292258
0.0028127511384945084
0.0028797214036967588
0.0029466916688990086
0.003013661934101259
0.00308063219930350

0.04480310742030538
0.04487007768550764
0.04493704795070989
0.045004018215912135
0.045070988481114384
0.04513795874631663
0.04520492901151889
0.045271899276721136
0.045338869541923385
0.045405839807125634
0.04547281007232789
0.04553978033753014
0.045606750602732386
0.045673720867934635
0.04574069113313689
0.04580766139833914
0.04587463166354139
0.045941601928743636
0.046008572193945885
0.04607554245914814
0.04614251272435039
0.04620948298955264
0.046276453254754886
0.04634342351995714
0.04641039378515939
0.04647736405036164
0.04654433431556389
0.04661130458076614
0.04667827484596839
0.04674524511117064
0.04681221537637289
0.046879185641575144
0.04694615590677739
0.04701312617197964
0.04708009643718189
0.04714706670238414
0.047214036967586394
0.04728100723278864
0.04734797749799089
0.04741494776319314
0.047481918028395395
0.047548888293597644
0.04761585855879989
0.04768282882400214
0.047749799089204396
0.047816769354406645
0.047883739619608894
0.04795070988481114
0.04801768015001339
0.0

0.07587731047414947
0.07594428073935172
0.07601125100455398
0.07607822126975623
0.07614519153495848
0.07621216180016073
0.07627913206536298
0.07634610233056523
0.07641307259576748
0.07648004286096972
0.07654701312617197
0.07661398339137424
0.07668095365657648
0.07674792392177873
0.07681489418698098
0.07688186445218323
0.07694883471738548
0.07701580498258773
0.07708277524778998
0.07714974551299224
0.07721671577819449
0.07728368604339673
0.07735065630859898
0.07741762657380123
0.07748459683900348
0.07755156710420573
0.07761853736940798
0.07768550763461023
0.07775247789981249
0.07781944816501474
0.07788641843021699
0.07795338869541923
0.07802035896062148
0.07808732922582373
0.07815429949102598
0.07822126975622823
0.07828824002143049
0.07835521028663274
0.07842218055183499
0.07848915081703724
0.07855612108223949
0.07862309134744173
0.07869006161264398
0.07875703187784623
0.07882400214304848
0.07889097240825074
0.07895794267345299
0.07902491293865524
0.07909188320385749
0.07915885346905974


0.12402893115456737
0.12409590141976962
0.12416287168497188
0.12422984195017413
0.12429681221537638
0.12436378248057862
0.12443075274578087
0.12449772301098312
0.12456469327618537
0.12463166354138762
0.12469863380658987
0.12476560407179213
0.12483257433699438
0.12489954460219663
0.12496651486739888
0.12503348513260112
0.12510045539780337
0.12516742566300562
0.12523439592820787
0.12530136619341012
0.12536833645861237
0.12543530672381462
0.12550227698901686
0.1255692472542191
0.1256362175194214
0.12570318778462364
0.1257701580498259
0.12583712831502814
0.12590409858023038
0.12597106884543263
0.12603803911063488
0.12610500937583713
0.12617197964103938
0.12623894990624163
0.12630592017144388
0.12637289043664612
0.12643986070184837
0.12650683096705062
0.12657380123225287
0.12664077149745512
0.12670774176265737
0.12677471202785964
0.1268416822930619
0.12690865255826414
0.1269756228234664
0.12704259308866864
0.12710956335387089
0.12717653361907313
0.12724350388427538
0.12731047414947763
0.127

0.15242432360032146
0.1524912938655237
0.15255826413072596
0.1526252343959282
0.15269220466113045
0.1527591749263327
0.15282614519153495
0.1528931154567372
0.15296008572193945
0.1530270559871417
0.15309402625234395
0.15316099651754622
0.15322796678274847
0.15329493704795072
0.15336190731315297
0.15342887757835522
0.15349584784355746
0.1535628181087597
0.15362978837396196
0.1536967586391642
0.15376372890436646
0.1538306991695687
0.15389766943477096
0.1539646396999732
0.15403160996517545
0.1540985802303777
0.15416555049557995
0.1542325207607822
0.15429949102598448
0.15436646129118672
0.15443343155638897
0.15450040182159122
0.15456737208679347
0.15463434235199572
0.15470131261719797
0.15476828288240022
0.15483525314760246
0.1549022234128047
0.15496919367800696
0.1550361639432092
0.15510313420841146
0.1551701044736137
0.15523707473881596
0.1553040450040182
0.15537101526922045
0.15543798553442273
0.15550495579962498
0.15557192606482723
0.15563889633002947
0.15570586659523172
0.1557728368604

...

0.8083980712563622
0.8084650415215644
0.8085320117867667
0.8085989820519689
0.8086659523171712
0.8087329225823734
0.8087998928475757
0.8088668631127779
0.8089338333779802
0.8090008036431824
0.8090677739083847
0.8091347441735869
0.8092017144387892
0.8092686847039914
0.8093356549691937
0.8094026252343959
0.8094695954995982
0.8095365657648004
0.8096035360300027
0.809670506295205
0.8097374765604072
0.8098044468256095
0.8098714170908117
0.809938387356014
0.8100053576212162
0.8100723278864185
0.8101392981516207
0.810206268416823
0.8102732386820252
0.8103402089472275
0.8104071792124297
0.810474149477632
0.8105411197428342
0.8106080900080365
0.8106750602732387
0.810742030538441
0.8108090008036432
0.8108759710688455
0.8109429413340477
0.81100991159925
0.8110768818644521
0.8111438521296545
0.8112108223948566
0.811277792660059
0.8113447629252611
0.8114117331904634
0.8114787034556656
0.8115456737208679
0.8116126439860701
0.8116796142512724
0.8117465845164746
0.8118135547816769
0.8118805250468791
0

0.8781810875971069
0.8782480578623091
0.8783150281275114
0.8783819983927136
0.8784489686579159
0.8785159389231182
0.8785829091883204
0.8786498794535227
0.8787168497187249
0.8787838199839272
0.8788507902491294
0.8789177605143317
0.8789847307795339
0.8790517010447362
0.8791186713099384
0.8791856415751407
0.8792526118403429
0.8793195821055452
0.8793865523707474
0.8794535226359497
0.8795204929011519
0.8795874631663542
0.8796544334315564
0.8797214036967587
0.8797883739619609
0.8798553442271632
0.8799223144923654
0.8799892847575677
0.8800562550227699
0.8801232252879722
0.8801901955531743
0.8802571658183767
0.8803241360835788
0.8803911063487811
0.8804580766139833
0.8805250468791856
0.8805920171443878
0.8806589874095901
0.8807259576747923
0.8807929279399946
0.880859898205197
0.8809268684703991
0.8809938387356014
0.8810608090008036
0.8811277792660059
0.8811947495312081
0.8812617197964104
0.8813286900616126
0.8813956603268149
0.8814626305920171
0.8815296008572194
0.8815965711224216
0.88166354138

0.9193678006964907
0.919434770961693
0.9195017412268952
0.9195687114920975
0.9196356817572997
0.919702652022502
0.9197696222877043
0.9198365925529065
0.9199035628181088
0.919970533083311
0.9200375033485133
0.9201044736137155
0.9201714438789178
0.92023841414412
0.9203053844093223
0.9203723546745245
0.9204393249397268
0.920506295204929
0.9205732654701313
0.9206402357353335
0.9207072060005358
0.920774176265738
0.9208411465309403
0.9209081167961425
0.9209750870613448
0.921042057326547
0.9211090275917493
0.9211759978569515
0.9212429681221538
0.921309938387356
0.9213769086525583
0.9214438789177605
0.9215108491829628
0.921577819448165
0.9216447897133673
0.9217117599785695
0.9217787302437718
0.921845700508974
0.9219126707741763
0.9219796410393785
0.9220466113045808
0.9221135815697831
0.9221805518349853
0.9222475221001876
0.9223144923653898
0.9223814626305921
0.9224484328957943
0.9225154031609966
0.9225823734261988
0.9226493436914011
0.9227163139566033
0.9227832842218056
0.9228502544870077
0.92

0.9527859630324136
0.9528529332976159
0.9529199035628181
0.9529868738280204
0.9530538440932226
0.9531208143584249
0.9531877846236271
0.9532547548888294
0.9533217251540316
0.9533886954192339
0.9534556656844361
0.9535226359496384
0.9535896062148406
0.9536565764800429
0.9537235467452451
0.9537905170104474
0.9538574872756496
0.9539244575408519
0.9539914278060541
0.9540583980712564
0.9541253683364586
0.9541923386016609
0.9542593088668632
0.9543262791320654
0.9543932493972677
0.9544602196624699
0.9545271899276722
0.9545941601928744
0.9546611304580767
0.9547281007232788
0.9547950709884812
0.9548620412536833
0.9549290115188857
0.9549959817840878
0.9550629520492901
0.9551299223144923
0.9551968925796946
0.9552638628448968
0.9553308331100991
0.9553978033753013
0.9554647736405036
0.9555317439057058
0.9555987141709081
0.9556656844361103
0.9557326547013126
0.9557996249665148
0.9558665952317171
0.9559335654969193
0.9560005357621216
0.9560675060273238
0.9561344762925261
0.9562014465577283
0.9562684168

0.9865389766943478
0.98660594695955
0.9866729172247523
0.9867398874899544
0.9868068577551568
0.986873828020359
0.9869407982855612
0.9870077685507634
0.9870747388159657
0.9871417090811679
0.9872086793463702
0.9872756496115724
0.9873426198767747
0.9874095901419769
0.9874765604071792
0.9875435306723814
0.9876105009375837
0.9876774712027859
0.9877444414679882
0.9878114117331904
0.9878783819983927
0.9879453522635949
0.9880123225287972
0.9880792927939994
0.9881462630592017
0.9882132333244039
0.9882802035896062
0.9883471738548084
0.9884141441200107
0.9884811143852129
0.9885480846504152
0.9886150549156175
0.9886820251808197
0.988748995446022
0.9888159657112242
0.9888829359764265
0.9889499062416287
0.989016876506831
0.9890838467720332
0.9891508170372355
0.9892177873024377
0.98928475756764
0.9893517278328422
0.9894186980980445
0.9894856683632467
0.989552638628449
0.9896196088936512
0.9896865791588535
0.9897535494240557
0.989820519689258
0.9898874899544602
0.9899544602196625
0.9900214304848647
0.

In [22]:
odds_dict

defaultdict(<function __main__.<lambda>()>,
            {2010: 0.5428571428571428,
             2011: 0.5080213903743316,
             2012: 0.4797979797979798,
             2013: 0.4846938775510204,
             2014: 0.4846938775510204,
             2015: 0.48223350253807107,
             2016: 0.4846938775510204,
             2017: 0.48223350253807107,
             2018: 0.5307262569832403,
             2019: 0.4797979797979798})

In [23]:
metric_dict['season_Supercoach']

defaultdict(<function __main__.<lambda>()>,
            {2010: 0.6845238095238095,
             2011: 0.6417112299465241,
             2012: 0.6717171717171717,
             2013: 0.6414141414141414,
             2014: 0.6262626262626263,
             2015: 0.6345177664974619,
             2016: 0.7070707070707071,
             2017: 0.5656565656565656,
             2018: 0.6166666666666667,
             2019: 0.5808080808080808})

Can we create a basic model now?

Firstly, we'll only use data from 2010 and later, so we can try to take advantage of all of the aggregated player data.

In [25]:
model_data = team_df[team_df['season'] >= 2010]

Need to set up a sampling strategy that doesn't bias across seasons or rounds. As such, we'll take a random sample from each round for each fold. We'll remove a quarter of the games per round for testing. Then we'll use 3-fold cross validation on the remainder.

In [26]:
# How many games do we get per round?
round_counts = {}
curr_round = 1
matches_in_round = 0
for idx,row in model_data.iterrows():
    
    if curr_round != row['round']:
        
        if matches_in_round not in round_counts:
            round_counts[matches_in_round] = 1
        else:
            round_counts[matches_in_round] += 1
            
        curr_round = row['round']
        matches_in_round = 1
        continue
        
    else:
        matches_in_round += 1
        
round_counts

{8: 44, 7: 7, 9: 154, 6: 22}

In [27]:
# At least 6 games in every round. Let's take out 25% of the games, minimum
from math import ceil
test_sample_size = {}
for num_games in round_counts:
    test_sample_size[num_games] = ceil(num_games/4)

In [28]:
test_sample_size

{8: 2, 7: 2, 9: 3, 6: 2}

In [29]:
from typing import Dict
import numpy as np

def get_season_rounds(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and number of games
    in season as values
    """
    seasons = matches['season'].unique()
    rounds_in_season = dict.fromkeys(seasons,0)
    
    for season in seasons:
        rounds_in_season[season] = max(matches[matches['season']==season]['round'])
    
    return rounds_in_season

# What teams participated in each season?
def get_season_teams(matches: pd.DataFrame) -> Dict:
    """
    Return a dictionary with seasons as keys and a list of teams who played
    in that season as values
    """
    seasons = matches['season'].unique()
    teams_in_season = {}

    for season in seasons:
        teams = list(matches[matches['season']==season]['hteam'].unique())
        teams.extend(list(matches[matches['season']==season]['ateam'].unique()))
        teams = np.unique(teams)
        teams_in_season[season] = list(teams)
        
    return teams_in_season

In [30]:
rounds_in_season = get_season_rounds(model_data)
teams_in_season = get_season_teams(model_data)

In [31]:
# Create the test set
from copy import deepcopy

test_data = pd.DataFrame()
for season, max_round in rounds_in_season.items():
    for rnd in range(1, max_round):
        round_matches = model_data[(model_data['season']==season) & (model_data['round']==rnd)]
        num_test = test_sample_size[len(round_matches)]
        round_test_set = round_matches.sample(num_test)
        test_data = test_data.append(round_test_set)

In [32]:
# Create the training set
training_data = model_data.drop(test_data.index)

In [33]:
model_data

,Unnamed: 0,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,...,a_season_disposal_efficiency,a_season_effective_disposals,a_season_intercepts,a_season_metres_gained,a_season_stoppage_clearances,a_season_score_involvements,a_season_tackles_in_50,a_season_turnovers,a_career_games_played,a_season_games_played
13004,13004,1,M.C.G.,2010-03-25,Thu,19:10:00,72010.0,Richmond,4.2.26,7.6.48,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1462.0,310.0
13005,13005,1,M.C.G.,2010-03-26,Fri,19:40:00,57772.0,Geelong,2.5.17,8.7.55,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1460.0,331.0
13006,13006,1,Gabba,2010-03-27,Sat,19:10:00,29201.0,Brisbane Lions,2.6.18,6.10.46,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1752.0,339.0
13007,13007,1,Stadium Australia,2010-03-27,Sat,19:10:00,31330.0,Sydney,3.3.21,7.5.47,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2372.0,442.0
13008,13008,1,M.C.G.,2010-03-27,Sat,14:10:00,45615.0,Melbourne,0.3.3,1.6.12,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1360.0,253.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14927,14927,23,Bellerive Oval,2019-08-24,Sat,14:10:00,8202.0,North Melbourne,3.4.22,5.4.34,...,1514.394425,4250.0,1179.0,94646.0,496.0,1356.0,183.0,1246.0,1999.0,334.0
14928,14928,23,S.C.G.,2019-08-24,Sat,13:45:00,33722.0,Sydney,6.2.38,6.3.39,...,1526.114099,4298.0,1040.0,88560.0,488.0,1432.0,171.0,1055.0,1976.0,341.0
14929,14929,23,Eureka Stadium,2019-08-25,Sun,13:10:00,9560.0,Footscray,6.5.41,9.8.62,...,1514.210238,4326.0,1122.0,83221.0,396.0,1364.0,139.0,1066.0,2788.0,315.0
14930,14930,23,Adelaide Oval,2019-08-25,Sun,16:10:00,27504.0,Port Adelaide,3.4.22,7.9.51,...,1502.111584,4160.0,1110.0,87477.0,455.0,1355.0,149.0,1140.0,1621.0,327.0


In [34]:
training_data

,Unnamed: 0,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,...,a_season_disposal_efficiency,a_season_effective_disposals,a_season_intercepts,a_season_metres_gained,a_season_stoppage_clearances,a_season_score_involvements,a_season_tackles_in_50,a_season_turnovers,a_career_games_played,a_season_games_played
13004,13004,1,M.C.G.,2010-03-25,Thu,19:10:00,72010.0,Richmond,4.2.26,7.6.48,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1462.0,310.0
13006,13006,1,Gabba,2010-03-27,Sat,19:10:00,29201.0,Brisbane Lions,2.6.18,6.10.46,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1752.0,339.0
13007,13007,1,Stadium Australia,2010-03-27,Sat,19:10:00,31330.0,Sydney,3.3.21,7.5.47,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2372.0,442.0
13009,13009,1,Football Park,2010-03-28,Sun,12:40:00,21205.0,Port Adelaide,5.4.34,10.10.70,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1434.0,312.0
13010,13010,1,Subiaco,2010-03-28,Sun,17:10:00,30976.0,Fremantle,2.5.17,9.9.63,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2120.0,358.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14927,14927,23,Bellerive Oval,2019-08-24,Sat,14:10:00,8202.0,North Melbourne,3.4.22,5.4.34,...,1514.394425,4250.0,1179.0,94646.0,496.0,1356.0,183.0,1246.0,1999.0,334.0
14928,14928,23,S.C.G.,2019-08-24,Sat,13:45:00,33722.0,Sydney,6.2.38,6.3.39,...,1526.114099,4298.0,1040.0,88560.0,488.0,1432.0,171.0,1055.0,1976.0,341.0
14929,14929,23,Eureka Stadium,2019-08-25,Sun,13:10:00,9560.0,Footscray,6.5.41,9.8.62,...,1514.210238,4326.0,1122.0,83221.0,396.0,1364.0,139.0,1066.0,2788.0,315.0
14930,14930,23,Adelaide Oval,2019-08-25,Sun,16:10:00,27504.0,Port Adelaide,3.4.22,7.9.51,...,1502.111584,4160.0,1110.0,87477.0,455.0,1355.0,149.0,1140.0,1621.0,327.0


In [35]:
test_data

,Unnamed: 0,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,...,a_season_disposal_efficiency,a_season_effective_disposals,a_season_intercepts,a_season_metres_gained,a_season_stoppage_clearances,a_season_score_involvements,a_season_tackles_in_50,a_season_turnovers,a_career_games_played,a_season_games_played
13005,13005,1,M.C.G.,2010-03-26,Fri,19:40:00,57772.0,Geelong,2.5.17,8.7.55,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1460.0,331.0
13008,13008,1,M.C.G.,2010-03-27,Sat,14:10:00,45615.0,Melbourne,0.3.3,1.6.12,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1360.0,253.0
13018,13018,2,M.C.G.,2010-04-04,Sun,16:40:00,27358.0,Richmond,2.0.12,4.5.29,...,1398.300000,249.0,0.0,0.0,0.0,0.0,0.0,0.0,2590.0,45.0
13012,13012,2,Gabba,2010-04-01,Thu,19:40:00,36780.0,Brisbane Lions,4.3.27,8.3.51,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1594.0,38.0
13021,13021,3,S.C.G.,2010-04-10,Sat,19:10:00,28414.0,Sydney,6.4.40,10.6.66,...,0.000000,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1444.0,104.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
14908,14908,21,Kardinia Park,2019-08-10,Sat,19:25:00,21664.0,Geelong,1.3.9,3.9.27,...,1599.646705,4563.0,1133.0,92591.0,417.0,1539.0,156.0,1120.0,2501.0,354.0
14912,14912,21,M.C.G.,2019-08-11,Sun,15:20:00,51039.0,Richmond,3.2.20,6.4.40,...,1539.163684,4015.0,1115.0,87658.0,481.0,1237.0,143.0,1126.0,2136.0,335.0
14916,14916,22,M.C.G.,2019-08-17,Sat,13:45:00,51786.0,Carlton,2.4.16,5.5.35,...,1600.119947,4174.0,1026.0,84668.0,461.0,1369.0,162.0,1011.0,1968.0,332.0
14919,14919,22,Docklands,2019-08-17,Sat,19:25:00,17063.0,North Melbourne,6.3.39,12.8.80,...,1524.187091,4337.0,1251.0,92713.0,460.0,1404.0,193.0,1213.0,2478.0,326.0


In [36]:
features = [col 
            for col 
            in ['h_career_' + col for col in player_cols_to_agg] + \
            ['h_season_' + col for col in player_cols_to_agg] + \
            ['a_career_' + col for col in player_cols_to_agg] + \
            ['a_season_' + col for col in player_cols_to_agg] + \
            ['h_' + col for col in ladder_columns] + \
            ['h_' + col + '_form' for col in ladder_columns] + \
            ['a_' + col for col in ladder_columns] + \
            ['a_' + col + '_form' for col in ladder_columns] + \
            ['h_career_' + col for col in misc_columns] + \
            ['h_season_' + col for col in misc_columns] + \
            ['a_career_' + col for col in misc_columns] + \
            ['a_season_' + col for col in misc_columns]
            ]

target = 'winner'

In [37]:
X_train = training_data[features]
y_train = training_data[target]
X_test = test_data[features]
y_test = test_data[target]

In [39]:
from sklearn.linear_model import LogisticRegressionCV
from sklearn.impute import SimpleImputer
X_train.fillna(0)
y_train.fillna(0)
log_reg = LogisticRegressionCV(cv=5,Cs=1)
log_reg.fit(X_train,y_train)

ValueError: Input contains NaN, infinity or a value too large for dtype('float64').

In [321]:
log_reg.score(X_train,y_train)

0.7561338289962826

In [322]:
log_reg.score(X_test,y_test)

0.7204116638078902

In [339]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import GridSearchCV
import numpy as np

clf = RandomForestClassifier() #Initialize with whatever parameters you want to

param_grid = {
                 'n_estimators': [100, 200, 500],
                 'max_depth': [5, 9, 20],
                 'min_samples_split': [2,3,4]
             }

grid_clf = GridSearchCV(clf, param_grid, cv=10)
grid_clf.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_split.py:657: Warning: The least populated class in y has only 9 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/model_selection/_search.py:814: DeprecationWarning: The default of the `iid` parameter will change from True to False in version 0.22 and will be removed in 0.24. This will change numeric results when test-set sizes are unequal.
  DeprecationWarning)


GridSearchCV(cv=10, error_score='raise-deprecating',
             estimator=RandomForestClassifier(bootstrap=True, class_weight=None,
                                              criterion='gini', max_depth=None,
                                              max_features='auto',
                                              max_leaf_nodes=None,
                                              min_impurity_decrease=0.0,
                                              min_impurity_split=None,
                                              min_samples_leaf=1,
                                              min_samples_split=2,
                                              min_weight_fraction_leaf=0.0,
                                              n_estimators='warn', n_jobs=None,
                                              oob_score=False,
                                              random_state=None, verbose=0,
                                              warm_start=False),
             ii

In [333]:
grid_clf.best_params_

{'max_depth': 9, 'n_estimators': 100}

In [340]:
grid_clf.score(X_train,y_train)

0.9628252788104089

In [341]:
grid_clf.score(X_test,y_test)

0.6792452830188679

In [343]:
from sklearn.linear_model import LogisticRegression
log_reg = LogisticRegression()

param_grid = {
                 'tol': [.0001, .001, .01],
                 'C': [.1, 1, 10],
                 'max_iter': [50,100,200]
             }

grid_log_reg = GridSearchCV(log_reg, param_grid, cv=5)
grid_log_reg.fit(X_train, y_train)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: De

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [50, 100, 200],
                         'tol': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [345]:
grid_log_reg.best_params_

{'C': 0.1, 'max_iter': 50, 'tol': 0.01}

In [346]:
grid_log_reg.score(X_train,y_train)

0.7695167286245354

In [347]:
grid_log_reg.score(X_test,y_test)

0.7444253859348199

In [310]:
model_data

,Unnamed: 0,round,venue,date,day,time,crowd,hteam,hteam_q1,hteam_q2,hteam_q3,hteam_q4,ateam,ateam_q1,ateam_q2,ateam_q3,ateam_q4,umpire1,umpire2,umpire3,umpire1games,umpire2games,umpire3games,hteam_et,ateam_et,season,matchid,homeodds,homeline,awayodds,awayline,hscore,ascore,hteam_q1_goals,hteam_q1_behinds,hteam_q1_score,hteam_q2_goals,hteam_q2_behinds,hteam_q2_score,hteam_q3_goals,hteam_q3_behinds,hteam_q3_score,hteam_q4_goals,hteam_q4_behinds,hteam_q4_score,ateam_q1_goals,ateam_q1_behinds,ateam_q1_score,ateam_q2_goals,ateam_q2_behinds,ateam_q2_score,ateam_q3_goals,ateam_q3_behinds,ateam_q3_score,ateam_q4_goals,ateam_q4_behinds,ateam_q4_score,winner,h_ladder_obj,a_ladder_obj,h_points_for,a_points_for,h_wins,a_wins,h_position,a_position,h_points_against,a_points_against,h_losses,a_losses,h_played,a_played,h_percentage,a_percentage,h_draws,a_draws,h_prem_points,a_prem_points,h_points_for_form,h_wins_form,h_position_form,h_points_against_form,h_losses_form,h_played_form,h_percentage_form,h_draws_form,h_prem_points_form,a_points_for_form,a_wins_form,a_position_form,a_points_against_form,a_losses_form,a_played_form,a_percentage_form,a_draws_form,a_prem_points_form,h_AFLfantasy,h_Supercoach,h_behinds,h_bounces,h_brownlow,h_clangers,h_clearances,h_contested_marks,h_contested_poss,h_disposals,h_frees_against,h_frees_for,h_goal_assists,h_goals,h_handballs,h_hitouts,h_inside50,h_kicks,h_marks,h_marks_in_50,h_one_percenters,h_rebound50,h_tackles,h_tog,h_uncontested_poss,h_centre_clearances,h_disposal_efficiency,h_effective_disposals,h_intercepts,h_metres_gained,h_stoppage_clearances,h_score_involvements,h_tackles_in_50,h_turnovers,h_career_AFLfantasy,h_career_Supercoach,h_career_behinds,h_career_bounces,h_career_brownlow,h_career_clangers,h_career_clearances,h_career_contested_marks,h_career_contested_poss,h_career_disposals,h_career_frees_against,h_career_frees_for,h_career_goal_assists,h_career_goals,h_career_handballs,h_career_hitouts,h_career_inside50,h_career_kicks,h_career_marks,h_career_marks_in_50,h_career_one_percenters,h_career_rebound50,h_career_tackles,h_career_tog,h_career_uncontested_poss,h_career_centre_clearances,h_career_disposal_efficiency,h_career_effective_disposals,h_career_intercepts,h_career_metres_gained,h_career_stoppage_clearances,h_career_score_involvements,h_career_tackles_in_50,h_career_turnovers,h_season_AFLfantasy,h_season_Supercoach,h_season_behinds,h_season_bounces,h_season_brownlow,h_season_clangers,h_season_clearances,h_season_contested_marks,h_season_contested_poss,h_season_disposals,h_season_frees_against,h_season_frees_for,h_season_goal_assists,h_season_goals,h_season_handballs,h_season_hitouts,h_season_inside50,h_season_kicks,h_season_marks,h_season_marks_in_50,h_season_one_percenters,h_season_rebound50,h_season_tackles,h_season_tog,h_season_uncontested_poss,h_season_centre_clearances,h_season_disposal_efficiency,h_season_effective_disposals,h_season_intercepts,h_season_metres_gained,h_season_stoppage_clearances,h_season_score_involvements,h_season_tackles_in_50,h_season_turnovers,h_career_games_played,h_season_games_played,a_AFLfantasy,a_Supercoach,a_behinds,a_bounces,a_brownlow,a_clangers,a_clearances,a_contested_marks,a_contested_poss,a_disposals,a_frees_against,a_frees_for,a_goal_assists,a_goals,a_handballs,a_hitouts,a_inside50,a_kicks,a_marks,a_marks_in_50,a_one_percenters,a_rebound50,a_tackles,a_tog,a_uncontested_poss,a_centre_clearances,a_disposal_efficiency,a_effective_disposals,a_intercepts,a_metres_gained,a_stoppage_clearances,a_score_involvements,a_tackles_in_50,a_turnovers,a_career_AFLfantasy,a_career_Supercoach,a_career_behinds,a_career_bounces,a_career_brownlow,a_career_clangers,a_career_clearances,a_career_contested_marks,a_career_contested_poss,a_career_disposals,a_career_frees_against,a_career_frees_for,a_career_goal_assists,a_career_goals,a_career_handballs,a_career_hitouts,a_career_inside50,a_career_kicks,a_career_marks,a_career_marks_in_50,a_career_one_percenters,a_career_rebound5

In [325]:
player_df

,Unnamed: 0,AFLfantasy,Supercoach,behinds,bounces,brownlow,clangers,clearances,contested_marks,contested_poss,disposals,first_name,frees_against,frees_for,fullkey,fullname,goal_assists,goals,handballs,hitouts,homeAway,inside50,kicks,last_name,marks,marks_in_50,matchid,namekey,number,one_percenters,playerid,rebound50,tackles,team,tog,uncontested_poss,centre_clearances,disposal_efficiency,effective_disposals,intercepts,metres_gained,stoppage_clearances,score_involvements,tackles_in_50,turnovers,season,round,round_num,season_AFLfantasy,career_AFLfantasy,season_Supercoach,career_Supercoach,season_behinds,career_behinds,season_bounces,career_bounces,season_brownlow,career_brownlow,season_clangers,career_clangers,season_clearances,career_clearances,season_contested_marks,career_contested_marks,season_contested_poss,career_contested_poss,season_disposals,career_disposals,season_frees_against,career_frees_against,season_frees_for,career_frees_for,season_goal_assists,career_goal_assists,season_goals,career_goals,season_handballs,career_handballs,season_hitouts,career_hitouts,season_inside50,career_inside50,season_kicks,career_kicks,season_marks,career_marks,season_marks_in_50,career_marks_in_50,season_one_percenters,career_one_percenters,season_rebound50,career_rebound50,season_tackles,career_tackles,season_tog,career_tog,season_uncontested_poss,career_uncontested_poss,season_centre_clearances,career_centre_clearances,season_disposal_efficiency,career_disposal_efficiency,season_effective_disposals,career_effective_disposals,season_intercepts,career_intercepts,season_metres_gained,career_metres_gained,season_stoppage_clearances,career_stoppage_clearances,season_score_involvements,career_score_involvements,season_tackles_in_50,career_tackles_in_50,season_turnovers,career_turnovers,career_games_played,season_games_played
0,555689,38.0,33.0,0.0,0.0,0.0,1.0,0.0,1.0,3.0,9.0,Aaron,0.0,0.0,201124NORRCHablack5,Aaron Black,2.0,1.0,4.0,0.0,Away,0.0,5.0,Black,3.0,2.0,201124NORRCH,ablack5,0.0,0.0,Aaron_Black,0.0,0.0,North Melbourne,52.0,6.0,0.0,66.7,6.0,0.0,0.0,0.0,0.0,0.0,0.0,2011,24,24,38.0,38.0,33.0,33.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,0.0,1.0,1.0,3.0,3.0,9.0,9.0,0.0,0.0,0.0,0.0,2.0,2.0,1.0,1.0,4.0,4.0,0.0,0.0,0.0,0.0,5.0,5.0,3.0,3.0,2.0,2.0,0.0,0.0,0.0,0.0,0.0,0.0,52.0,52.0,6.0,6.0,0.0,0.0,66.700000,66.700000,6.0,6.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1,1
1,559364,55.0,85.0,0.0,0.0,0.0,1.0,0.0,1.0,7.0,11.0,Aaron,1.0,0.0,201209BRSNORablack5,Aaron Black,2.0,3.0,6.0,0.0,Home,0.0,5.0,Black,3.0,3.0,201209BRSNOR,ablack5,35.0,0.0,Aaron_Black,0.0,1.0,North Melbourne,82.0,4.0,0.0,72.7,8.0,0.0,0.0,0.0,0.0,0.0,0.0,2012,09,9,55.0,93.0,85.0,118.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,1.0,2.0,7.0,10.0,11.0,20.0,1.0,1.0,0.0,0.0,2.0,4.0,3.0,4.0,6.0,10.0,0.0,0.0,0.0,0.0,5.0,10.0,3.0,6.0,3.0,5.0,0.0,0.0,0.0,0.0,1.0,1.0,82.0,134.0,4.0,10.0,0.0,0.0,72.700000,69.700000,8.0,14.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2,1
2,560027,41.0,41.0,1.0,0.0,0.0,0.0,0.0,1.0,4.0,8.0,Aaron,0.0,0.0,201210HAWNORablack4,Aaron Black,0.0,0.0,4.0,0.0,Away,2.0,4.0,Black,4.0,0.0,201210HAWNOR,ablack4,35.0,0.0,Aaron_Black,0.0,2.0,North Melbourne,83.0,4.0,0.0,62.5,5.0,0.0,0.0,0.0,0.0,0.0,0.0,2012,10,10,96.0,134.0,126.0,159.0,1.0,1.0,0.0,0.0,0.0,0.0,1.0,2.0,0.0,0.0,2.0,3.0,11.0,14.0,19.0,28.0,1.0,1.0,0.0,0.0,2.0,4.0,3.0,4.0,10.0,14.0,0.0,0.0,2.0,2.0,9.0,14.0,7.0,10.0,3.0,5.0,0.0,0.0,0.0,0.0,3.0,3.0,165.0,217.0,8.0,14.0,0.0,0.0,67.600000,67.300000,13.0,19.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,3,2
3,560538,21.0,23.0,0.0,0.0,0.0,1.0,0.0,0.0,3.0,4.0,Aaron,1.0,0.0,201212GCSNORablack1,Aaron Black,0.0,0.0,3.0,0.0,Away,0.0,1.0,Black,1.0,0.0,201212GCSNOR,ablack1,0.0,1.0,Aaron_Black,0.0,3.0,North Melbourne,53.0,1.0,0.0,75.0,3.0,0.0,0.0,0.0,0.0,0.0,0.0,2012,12,12,117.0,155.0,149.0,182.0,1.0,1.0,0.0,0.0,0.0,0.0,2.0,3.0,0.0,0.0,2.0,3.0,14.0,17.0,23.0,32.0,2.0,2.0,0.0,0.0,2.0,4.0,3.0,4.0,13.0,17.0,0.0,0.0,2.0,2.0,10.0,15.0,8.0,11.0,3.0,5.0,1.0,1.0,0.0,0.0,6.0,6.0,2

How would this have performed in 2019?

In [354]:
data_2019 = model_data[model_data['season']==2019]
data_not_2019 = model_data[model_data['season']!=2019]
X_train_2019 = data_not_2019[features]
y_train_2019 = data_not_2019[target]
X_test_2019 = data_2019[features]
y_test_2019 = data_2019[target]

In [351]:
from sklearn.linear_model import LogisticRegression
log_reg_2019 = LogisticRegression()

param_grid = {
                 'tol': [.0001, .001, .01],
                 'C': [.1, 1, 10],
                 'max_iter': [50,100,200]
             }

grid_log_reg_2019 = GridSearchCV(log_reg_2019, param_grid, cv=5)
grid_log_reg_2019.fit(X_train_2019, y_train_2019)

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: De

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/svm/base.py:929: ConvergenceWarning: Liblinear failed to converge, increase the number of iterations.
  "the number of iterations.", ConvergenceWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Defa

/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:432: FutureWarning: Default solver will be changed to 'lbfgs' in 0.22. Specify a solver to silence this warning.
  FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_model/logistic.py:469: FutureWarning: Default multi_class will be changed to 'auto' in 0.22. Specify the multi_class option to silence this warning.
  "this warning.", FutureWarning)
/Applications/anaconda3/lib/python3.7/site-packages/sklearn/linear_mod

GridSearchCV(cv=5, error_score='raise-deprecating',
             estimator=LogisticRegression(C=1.0, class_weight=None, dual=False,
                                          fit_intercept=True,
                                          intercept_scaling=1, l1_ratio=None,
                                          max_iter=100, multi_class='warn',
                                          n_jobs=None, penalty='l2',
                                          random_state=None, solver='warn',
                                          tol=0.0001, verbose=0,
                                          warm_start=False),
             iid='warn', n_jobs=None,
             param_grid={'C': [0.1, 1, 10], 'max_iter': [50, 100, 200],
                         'tol': [0.0001, 0.001, 0.01]},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=0)

In [352]:
grid_log_reg_2019.score(X_train_2019, y_train_2019)

0.7635838150289017

In [355]:
grid_log_reg_2019.score(X_test_2019, y_test_2019)

0.6666666666666666

In [309]:
player_cols_to_agg = [
    'AFLfantasy',
    'Supercoach',
    'behinds',
    'bounces',
    'brownlow',
    'clangers',
    'clearances',
    'contested_marks',
    'contested_poss',
    'disposals',
    'frees_against',
    'frees_for',
    'goal_assists',
    'goals',
    'handballs',
    'hitouts',
    'inside50',
    'kicks',
    'marks',
    'marks_in_50',
    'one_percenters',
    'rebound50',
    'tackles',
    'tog',
    'uncontested_poss',
    'centre_clearances',
    'disposal_efficiency',
    'effective_disposals',
    'intercepts',
    'metres_gained',
    'stoppage_clearances',
    'score_involvements',
    'tackles_in_50',
    'turnovers'
]

match_cols = [
    'odds',
    'line'
]

ladder_columns = [
    'wins',
    'losses',
    'draws',
    'prem_points',
    'played',
    'points_for',
    'points_against',
    'percentage',
    'position'
]

misc_columns = [
    'games_played'
]

In [61]:
x = ['a','b','c']

In [56]:
x.remove('c')

In [57]:
x

['a']

In [69]:
from copy import deepcopy
y = deepcopy(x)
y.remove('a')

In [70]:
y

['b', 'c']

In [71]:
x

['a', 'b', 'c']

In [73]:
results.columns

NameError: name 'results' is not defined